In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("./data/final_filtered.csv", index_col=0)

In [ ]:
df["BioSample"]

/tmp/ipykernel_265406/2570026764.py:1: DtypeWarning: Columns (38,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  sandpiper_metadata = pd.read_csv(f"{data_dir}/sandpiper_metadata.csv", index_col=0)


In [5]:
marine = df[df["ScientificName"] == "marine metagenome"]

In [6]:
sp = pd.read_csv("data/sandpiper0.3.0.condensed.csv", sep="\t", index_col=0)

In [7]:
sp_marine = sp.merge(marine, left_index=True, right_index=True, how="inner")

In [8]:
counts_marine = sp_marine[["coverage", "taxonomy"]].copy()

In [9]:
lineage_marine = counts_marine['taxonomy'].str.split("; ", expand=True).iloc[:, 1:]

In [10]:
lineage_marine.columns = [
    "domain",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species",
]

In [11]:
# counts_marine = counts_marine.merge(
#     lineage_marine, left_index=True, right_index=True, how="inner")

In [13]:
lineage_marine["coverage"] = counts_marine["coverage"]

In [ ]:
lineage_marine.head(100000).pivot_table(
    index=["domain", "phylum", "class", "order", "family", "genus", "species"],
    values="coverage",
    aggfunc="sum",
)

coverage
domain      phylum                 class                 order                   family                 genus              species                                 
d__Archaea  p__Asgardarchaeota     c__Heimdallarchaeia   o__Heimdallarchaeales   f__Kariarchaeaceae     g__Kariarchaeum    s__Kariarchaeum pelagium           41.93
            p__Hydrothermarchaeota c__Hydrothermarchaeia o__Hydrothermarchaeales f__BMS3B               g__BMS3B           s__BMS3B sp002897955               18.34
                                                                                 f__SZUA-236            g__SZUA-236        s__SZUA-236 sp002923215            48.28
            p__Nanoarchaeota       c__Nanoarchaeia       o__Pacearchaeales       f__GW2011-AR1          g__BMS3ABIN17      s__BMS3ABIN17 sp002897655          64.18
                                                         o__UBA10117             f__ARS21               g__ARS21           s__ARS21 sp002686215                0.53
...                                                                                                                                                             ...
d__Bacteria p__Verrucomicrobiota   c__Verrucomicrobiae   o__Verrucomicrobiales   f__DEV007              g__EC70            s__EC70 sp028033615                 0.98
                                                                                                                           s__EC70 sp902510815                 0.99
                                                                                                        g__JAACDM01        s__JAACDM01 sp009936795            15.40
                                                                                 f__Verrucomicrobiaceae g__Prosthecobacter s__Prosthecobacter sp947507875      1.29
            p__Zixibacteria        c__MSB-5A5            o__GN15                 f__FEB-12              g__QQUK01          s__QQUK01 sp913009765              11.42

[1840 rows x 1 columns]

In [24]:
ranks = lineage_marine.columns[:-1].to_list()

In [ ]:
rank_dfs = dict() 
for rank in ranks:
    rank_dfs[rank] = lineage_marine.reset_index().pivot_table(
    index="sample",
    columns=rank,
    values="coverage",
    aggfunc="sum",
).fillna(0.0).astype(float)

In [28]:
for rank, df in rank_dfs.items():
    df.to_csv(f"data/rank_{rank}.csv", sep="\t", index=True, header=True)

In [31]:
gtdb_md = pd.read_csv("data/gtdb/bac120_metadata_r220.tsv", sep="\t", index_col=0)

In [39]:
sp_marine_species = rank_dfs["species"].columns.to_list()

In [40]:
gtdb_tax = gtdb_md["gtdb_taxonomy"].str.split(";", expand=True)

In [46]:
gtdb_species = gtdb_tax[6].to_list()

In [ ]:
intersection = [i for i in sp_marine_species if i in gtdb_species]

['s__0-14-0-80-60-11 sp016875015',
 's__0-14-0-80-60-11 sp018897875',
 's__0-14-0-80-60-11 sp030655955',
 's__01-FULL-49-22b sp913054235',
 's__1-14-2-50-31-20 sp018818165',
 's__1-19 sp002837195',
 's__12-FULL-67-14b sp002737365',
 's__12-FULL-67-14b sp030697165',
 's__12-FULL-67-14b sp945866285',
 's__2-02-FULL-66-22 sp029580855',
 's__2-12-FULL-35-11 sp029213075',
 's__2-12-FULL-35-15 sp016786925',
 's__2-12-FULL-35-15 sp017303895',
 's__2-12-FULL-35-15 sp945885475',
 's__2-12-FULL-36-22 sp018660415',
 's__2-12-FULL-64-23 sp016649475',
 's__2011-GWC2-44-17 sp001790515',
 's__28-YEA-48 sp029979185',
 's__28-YEA-48 sp903847605',
 's__28-YEA-48 sp903918155',
 's__28-YEA-48 sp945878875',
 's__28-YEA-48 sp947476075',
 's__34-128 sp000405345',
 's__34-128 sp000405605',
 's__34-128 sp002084865',
 's__34-128 sp003451675',
 's__34-128 sp003527405',
 's__34-128 sp014894735',
 's__41-12-T18 sp002163085',
 's__41-12-T18 sp002163115',
 's__41-12-T18 sp002400805',
 's__42-54-T18 sp002162955',
 's

In [49]:
intersection.__len__()

14184

In [50]:
sp_marine_species.__len__()

15290